# Twitter Named Entity Recognition Case Study

### About
Twitter is a microblogging and social networking service on which users post and interact with messages known as "tweets". Every second, on average, around 6,000 tweets are tweeted on Twitter, corresponding to over 350,000 tweets sent per minute, 500 million tweets per day.

### Problem statement 
Twitter wants to automatically tag and analyze tweets for better understanding of the trends and topics without being dependent on the hashtags that the users use. Many users do not use hashtags or sometimes use wrong or mis-spelled tags, so they want to completely remove this problem and create a system of recognizing important content of the tweets.

### Objective
Named Entity Recognition (NER) is an important subtask of information extraction that seeks to locate and recognise named entities.
We need to train models that will be able to identify the various named entities.

### Data
Dataset is annotated with 10 fine-grained NER categories: person, geo-location, company, facility, product,music artist, movie, sports team, tv show and other. Dataset was extracted from tweets and is structured in CoNLL format., in English language. Containing in Text file format.
The CoNLL format is a text file with one word per line with sentences separated by an empty line. The first word in a line should be the word and the last word should be the label.

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [23]:
import os
root_path = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
data_path = os.path.join(root_path,'data')
train_data_path = os.path.join(data_path,'wnut 16.txt.conll')
test_data_path = os.path.join(data_path,'wnut 16test.txt.conll')

## Getting the data

In [50]:
# reading the training file
with open(train_data_path,'r') as f:
    train_raw = f.read()
with open(test_data_path,'r') as f:
    test_raw = f.read()

In [51]:
# creating a function to format the data
def extract_ner_from_conll(conll_data):
    # Split the data into sentences based on empty lines
    sentences = [sentence.strip() for sentence in conll_data.strip().split('\n\n')]
    ner_data = []

    for sentence in sentences:
        tokenised_sentence = []
        for token_entity in sentence.split('\n'):
            token, entity = token_entity.split('\t')
            tokenised_sentence.append((token,entity))
        ner_data.append(tokenised_sentence)

    return ner_data

In [52]:
# preprocessing the raw files
train_data = extract_ner_from_conll(train_raw)
test_data = extract_ner_from_conll(test_raw)

In [68]:
# checking sentences after preprocessing
print(train_data[0])

[('@SammieLynnsMom', 'O'), ('@tg10781', 'O'), ('they', 'O'), ('will', 'O'), ('be', 'O'), ('all', 'O'), ('done', 'O'), ('by', 'O'), ('Sunday', 'O'), ('trust', 'O'), ('me', 'O'), ('*wink*', 'O')]


## Data preparation

In [61]:
# create a function to prepare the data to be fed into the model
def prepare_data(text_data):
    
    # initialize empty lists for sentences and entities
    sentences = []
    entities = []
    
    for sentence in text_data:
        
        # initialize empty lists for sentence text and corresponding entities
        word_list = []
        entity_list = []
        
        for token in sentence:
            word_list.append(token[0])
            entity_list.append(token[1])
        
        sentences.append(word_list)
        entities.append(entity_list)
    
    return (sentences,entities)

In [62]:
# call the prepare_data function on the train and test data
xtrain, ytrain = prepare_data(train_data)
xtest, ytest = prepare_data(test_data)

In [67]:
# checking sentences are conversion
print(xtrain[0])
print(ytrain[0])

['@SammieLynnsMom', '@tg10781', 'they', 'will', 'be', 'all', 'done', 'by', 'Sunday', 'trust', 'me', '*wink*']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
